In [1]:
import sqlite3
import pandas as pd
import yfinance as yf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split

#### Data Model Implementation (25 points)

In [ ]:
# The data is cleaned, normalized, and standardized prior to modeling (5 points)
msft = yf.Ticker('MSFT')
hist = msft.history('1y')
hist[:5]

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-09-18 00:00:00-04:00,325.357807,327.938442,323.928532,326.608429,16834200,0.0,0.0
2023-09-19 00:00:00-04:00,323.739973,326.935984,322.092337,326.201477,16505900,0.0,0.0
2023-09-20 00:00:00-04:00,327.055125,327.134516,318.122176,318.380219,21436500,0.0,0.0
2023-09-21 00:00:00-04:00,316.881498,322.926123,312.653225,317.149475,35529500,0.0,0.0
2023-09-22 00:00:00-04:00,318.926111,319.055147,313.794615,314.648224,21447900,0.0,0.0


In [ ]:
# dropped Volume, Dividends, and Stock Splits
df = hist.drop(['Volume','Dividends','Stock Splits'],axis=1)

# changing date format
df.index = df.index.strftime('%Y-%m-%d %a')

# Dropping the first two rows to start on a Monday
df = df[2:]

In [ ]:
# create and connect to sqlite database stock_db
db = sqlite3.connect('stock_db.sqlite')

In [ ]:
# Create or replace tesla table and load df data to it.
df.to_sql('msft',db, if_exists='replace')

249

In [ ]:
# The model utilizes data retrieved from SQL or Spark (5 points)
# read table tesla from stock_db
df = pd.read_sql('SELECT * FROM msft', db, index_col='Date')
df[:5]

,Open,High,Low,Close
Date,,,,
2023-09-20 Wed,327.055125,327.134516,318.122176,318.380219
2023-09-21 Thu,316.881498,322.926123,312.653225,317.149475
2023-09-22 Fri,318.926111,319.055147,313.794615,314.648224
2023-09-25 Mon,314.231351,315.303322,312.653201,315.174286
2023-09-26 Tue,312.782259,313.526672,307.710313,309.814545


In [ ]:
# Generating labels. Since I'm generating a label based on the next day's information, I need to have one less label.
print("Generating labels. Since I'm generating a label based on the next day's information, I need to have one less label.")
print(f'rows on dataframe: {len(df)}')

labels = []
for i in range(len(df)-1):
    today = df.iloc[i]
    tomorrow = df.iloc[i+1]

    if today.Open<tomorrow.Open:
        labels.append('Buy')
    else:
        labels.append('Sell')

print(f'Count of labels: {len(labels)}')

Generating labels. Since I'm generating a label based on the next day's information, I need to have one less label.
rows on dataframe: 249
Count of labels: 248


In [ ]:
# A Python script initializes, trains, and evaluates a model (10 points)
# I need to drop last row of data so lables can match
df = df[:-1].copy()

# Adding labels to dataframe
df['Recomendation'] = labels

# displaying dataframe
df

,Open,High,Low,Close,Recomendation
Date,,,,,
2023-09-20 Wed,327.055125,327.134516,318.122176,318.380219,Sell
2023-09-21 Thu,316.881498,322.926123,312.653225,317.149475,Buy
2023-09-22 Fri,318.926111,319.055147,313.794615,314.648224,Sell
2023-09-25 Mon,314.231351,315.303322,312.653201,315.174286,Sell
2023-09-26 Tue,312.782259,313.526672,307.710313,309.814545,Sell
...,...,...,...,...,...
2024-09-09 Mon,407.239990,408.649994,402.149994,405.720001,Buy
2024-09-10 Tue,408.200012,416.329987,407.700012,414.200012,Buy
2024-09-11 Wed,415.500000,423.989990,409.579987,423.040009,Buy


In [ ]:
# The data is cleaned, normalized, and standardized prior to modeling (5 points)
# splitting data into X and y
X = df.drop('Recomendation',axis=1).values
y = df.Recomendation.values

In [ ]:
X[:5]

array([[327.05512517, 327.13451584, 318.12217613, 318.38021851],
       [316.88149752, 322.92612291, 312.65322517, 317.1494751 ],
       [318.92611112, 319.05514744, 313.79461529, 314.64822388],
       [314.23135142, 315.30332221, 312.6532008 , 315.17428589],
       [312.78225923, 313.52667166, 307.71031342, 309.81454468]])

In [ ]:
y[:5]

array(['Sell', 'Buy', 'Sell', 'Sell', 'Sell'], dtype=object)

In [ ]:
# Tokenizing labels
new_y = [ 1 if v=='Sell' else 0 for v in y]
new_y[:5]

[1, 0, 1, 1, 1]

In [ ]:
# Split data into training and validation
X_train, X_test, y_train, y_test = train_test_split(X,new_y)

In [ ]:
# The model demonstrates meaningful predictive power at least 75%
# classification accuracy or 0.80 R-squared. (5 points)

model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [ ]:
# Prediction score
f'The validation accuracy score of {model.score(X_test,y_test)*100:.2f}% does not meet standards.'

'The validation accuracy score of 52.91% does not meet standards.'

#### Data Model Optimization (25 points)

In [ ]:
new_y = [ 'Buy' if x>=0 else 'Sell' for x in df.Close - df.Open]
new_y[:5]

['Sell', 'Buy', 'Sell', 'Buy', 'Sell']

In [ ]:
X[:5]

array([[327.05512517, 327.13451584, 318.12217613, 318.38021851],
       [316.88149752, 322.92612291, 312.65322517, 317.1494751 ],
       [318.92611112, 319.05514744, 313.79461529, 314.64822388],
       [314.23135142, 315.30332221, 312.6532008 , 315.17428589],
       [312.78225923, 313.52667166, 307.71031342, 309.81454468]])

In [ ]:
new_y = [1 if x=='Sell' else 0 for x in new_y]

In [ ]:
# Split data into training and validation
X_train, X_test, y_train, y_test = train_test_split(X,new_y)

In [ ]:
model2 = LinearRegression()
model2.fit(X_train,y_train)

LinearRegression()

In [ ]:
model2.score(X_train,y_train)

0.6216450333678483

### The linear model was not good enough.  I'm trying a Recurrent Neural Network

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, SimpleRNN

In [8]:
df = pd.read_csv('msft_data.csv', index_col='Date')
df = df[3:].copy()
df[:5]

,Open,High,Low,Close,Recomendation
Date,,,,,
2023-09-25 Mon,314.231351,315.303322,312.653201,315.174286,Sell
2023-09-26 Tue,312.782259,313.526672,307.710313,309.814545,Sell
2023-09-27 Wed,309.973286,311.958385,307.382745,310.459656,Sell
2023-09-28 Thu,308.673063,313.129633,307.144558,311.303345,Buy
2023-09-29 Fri,315.382713,317.089900,312.633361,313.397614,Sell


In [9]:
df.drop('Recomendation', axis=1, inplace=True)

In [13]:
import numpy as np

In [17]:
msft_data = np.array(df).reshape(-1,5,4)

In [19]:
df[:5]

,Open,High,Low,Close
Date,,,,
2023-09-25 Mon,314.231351,315.303322,312.653201,315.174286
2023-09-26 Tue,312.782259,313.526672,307.710313,309.814545
2023-09-27 Wed,309.973286,311.958385,307.382745,310.459656
2023-09-28 Thu,308.673063,313.129633,307.144558,311.303345
2023-09-29 Fri,315.382713,317.089900,312.633361,313.397614


In [21]:
msft_data[0]

array([[314.23135142, 315.30332221, 312.6532008 , 315.17428589],
       [312.78225923, 313.52667166, 307.71031342, 309.81454468],
       [309.97328577, 311.95838534, 307.38274537, 310.45965576],
       [308.67306343, 313.12963279, 307.1445585 , 311.30334473],
       [315.38271322, 317.08990016, 312.63336105, 313.39761353]])

In [ ]:
df

In [28]:
new_labels = []

for x in msft_data:
  open_mon = x[0][0]
  close_fri = x[-1][-1]

  if close_fri < open_mon:
    new_labels.append('Sell')
  else:
    new_labels.append('Buy')

In [33]:
y = [ 1 if x == 'Sell' else 0 for x in new_labels]

In [34]:
msft_data.shape

(49, 5, 4)

(5, 4)

In [ ]:
nn = Sequential()
nn.add(Input(shape=(msft_data.shape[1],msft_data.shape[2])))


In [35]:
?Input